# CEBD1260: Preprocessing pipeline assignment
*by Hamza Benhmani & Pierre-Olivier Bonin*

> In this notebook, we use our custom function `utils.py` to aggregate, merge, and fill missing values of the Credit Risk datasets.
> Please look [here](https://github.com/gzork/the-big-short/blob/main/utils.py) for the function.

In [1]:
import pandas as pd
import numpy as np
import glob
import ntpath
import re
from utils import *

data_dir = "D:/Documents/Pierre-Olivier/CEBD1260/Datasets/"

In [2]:
%%time
names = [re.sub('\.csv$', '', ntpath.basename(p)) for p in glob.glob(data_dir + "*.csv")]

dfs = {}
for df_name in names:
    dfs[df_name] = pd.read_csv(f"{data_dir}{df_name}.csv")

dfs["application_train"].shape

Wall time: 25.8 s


(307511, 122)

In [3]:
%%time
bureau_balance = dfs["bureau_balance"].reset_index().groupby(['SK_ID_BUREAU', 'STATUS'])['MONTHS_BALANCE'].aggregate(lambda x: x.max()-x.min()).unstack()
bureau_balance.columns = [f"{bureau_balance.columns.name}_{c}_MONTHS_BALANCE_MAX_MIN_DIFF" for c in bureau_balance.columns]
bureau_balance.reset_index(level=0, inplace=True)
bureau_balance

Wall time: 1min 7s


,SK_ID_BUREAU,STATUS_0_MONTHS_BALANCE_MAX_MIN_DIFF,STATUS_1_MONTHS_BALANCE_MAX_MIN_DIFF,STATUS_2_MONTHS_BALANCE_MAX_MIN_DIFF,STATUS_3_MONTHS_BALANCE_MAX_MIN_DIFF,STATUS_4_MONTHS_BALANCE_MAX_MIN_DIFF,STATUS_5_MONTHS_BALANCE_MAX_MIN_DIFF,STATUS_C_MONTHS_BALANCE_MAX_MIN_DIFF,STATUS_X_MONTHS_BALANCE_MAX_MIN_DIFF
0,5001709,NaN,NaN,NaN,NaN,NaN,NaN,85.0,10.0
1,5001710,5.0,NaN,NaN,NaN,NaN,NaN,47.0,33.0
2,5001711,2.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,5001712,9.0,NaN,NaN,NaN,NaN,NaN,8.0,NaN
4,5001713,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.0
...,...,...,...,...,...,...,...,...,...
817390,6842884,8.0,NaN,NaN,NaN,NaN,NaN,19.0,27.0
817391,6842885,11.0,NaN,NaN,NaN,NaN,11.0,NaN,NaN
817392,6842886,7.0,NaN,NaN,NaN,NaN,NaN,24.0,NaN
817393,6842887,5.0,NaN,NaN,NaN,NaN,NaN,30.0,NaN


In [4]:
%%time
dfs["bureau"] = dfs["bureau"].merge(bureau_balance,on='SK_ID_BUREAU',how='left')
for f in [f for f in dfs["bureau"].columns if dfs["bureau"][f].isnull().sum()>0]:
    dfs["bureau"][f] = dfs["bureau"][f].fillna(dfs["bureau"][f].median())

Wall time: 1.43 s


In [ ]:
%%time
aggr_dicts = {
    "POS_CASH_balance" : {
        'SK_DPD':['max','std','median'],
        'SK_DPD_DEF':['max','std','median'],
    },
    "credit_card_balance" : {
        'AMT_BALANCE':['max','std','median'],
        'AMT_CREDIT_LIMIT_ACTUAL':['max','std','median'],
        'AMT_PAYMENT_CURRENT':['max','std','median'],
        'AMT_RECEIVABLE_PRINCIPAL':['max','std','median'],
        'AMT_RECIVABLE':['max','std','median'],
        'CNT_INSTALMENT_MATURE_CUM':['max','std','median'],
        'SK_DPD':['max','std','median'],
        'SK_DPD_DEF':['max','std','median'],
    },
    "previous_application" : {
        'AMT_ANNUITY':['max','std','median'],
        'AMT_APPLICATION':['max','std','median'],
        'AMT_CREDIT':['max','std','median'],
        'AMT_DOWN_PAYMENT':['max','std','median'],
        'AMT_GOODS_PRICE':['max','std','median'],
        'RATE_DOWN_PAYMENT':['max','std','median'],
        'RATE_INTEREST_PRIMARY':['max','std','median'],
        'RATE_INTEREST_PRIVILEGED':['max','std','median'],
        'DAYS_DECISION':['max','std','median'],
    },
    "installments_payments" : {
        'AMT_INSTALMENT':['max','std','median'],
        'AMT_PAYMENT':['max','std','median'],
    },
    "bureau" : {
        'SK_ID_BUREAU':['nunique'],
        'CREDIT_ACTIVE':['nunique'],
        'CREDIT_CURRENCY':['nunique'],
        'DAYS_CREDIT':[lambda x: (x.max()-x.min(), "max_min_diff")],
        'CREDIT_DAY_OVERDUE':['sum', 'median'],
        'DAYS_CREDIT_ENDDATE':['std'],
        'DAYS_ENDDATE_FACT':['std'],
        'AMT_CREDIT_MAX_OVERDUE':['max'],
        'CNT_CREDIT_PROLONG':['sum', 'median'],
        'AMT_CREDIT_SUM':['sum', 'median'],
        'AMT_CREDIT_SUM_DEBT':['sum', 'median'],
        'AMT_CREDIT_SUM_LIMIT':['sum', 'median'],
        'AMT_CREDIT_SUM_OVERDUE':['sum', 'median'],
        'CREDIT_TYPE':['nunique'],
        'DAYS_CREDIT_UPDATE':['mean'],
        'AMT_ANNUITY':['sum', 'median'],
        'STATUS_0_MONTHS_BALANCE_MAX_MIN_DIFF':['mean'],
        'STATUS_1_MONTHS_BALANCE_MAX_MIN_DIFF':['mean'],
        'STATUS_2_MONTHS_BALANCE_MAX_MIN_DIFF':['mean'],
        'STATUS_3_MONTHS_BALANCE_MAX_MIN_DIFF':['mean'],
        'STATUS_4_MONTHS_BALANCE_MAX_MIN_DIFF':['mean'],
        'STATUS_5_MONTHS_BALANCE_MAX_MIN_DIFF':['mean'],
        'STATUS_C_MONTHS_BALANCE_MAX_MIN_DIFF':['mean'],
        'STATUS_X_MONTHS_BALANCE_MAX_MIN_DIFF':['mean'],
    }
}

main_df = dfs["application_train"]
for table in aggr_dicts:
    main_df = merge_with_aggr(main_df, dfs[table], "SK_ID_CURR", aggr_dicts[table], table)

main_df.shape, main_df.columns.tolist()

In [ ]:
main_df.head()

In [ ]:
main_df.isnull().sum().sum()/216

In [ ]:
na_numfiller(main_df, aggregation_func="median")

In [ ]:
main_df.isnull().sum().sum()

In [ ]:
na_catfiller(main_df)

In [ ]:
main_df.isnull().sum().sum()

In [ ]:
optimize_inttypes(main_df, specify="int32")

In [ ]:
optimize_floattypes(main_df, specify="float32")